In [ ]:
! python3.7 -m pip install -q braindecode moabb mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.0/365.0 kB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Carregamento dos dados

In [1]:
from skorch.helper import predefined_split
from skorch.callbacks import LRScheduler
from braindecode import EEGClassifier
from braindecode.datasets import MOABBDataset

subject_id = 3
dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=[subject_id])

/home/matias/Mestrado/venv_BCI/venv_BCI/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]


## Preprocessamento

In [2]:
from braindecode.preprocessing import preprocess, Preprocessor
import numpy as np
from numpy import multiply

low_cut_hz = 4.
high_cut_hz = 38.
newfreq = 128
factor = 1e6

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),
    Preprocessor(lambda data: multiply(data, factor)),
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),
    Preprocessor("set_eeg_reference", ref_channels="average", ch_type="eeg")
]

preprocess(dataset, preprocessors, n_jobs=-1);

/home/matias/Mestrado/venv_BCI/venv_BCI/lib/python3.7/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
Filtering raw data in 1 contiguous segment
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
Setting up band-pass filter from 4 - 38 Hz
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation

- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutof

In [3]:
from braindecode.preprocessing import create_windows_from_events

sfreq = dataset.datasets[0].raw.info['sfreq']

trial_start_offset_seconds = -0.5
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)
trial_stop_offset_seconds = -2
trial_stop_offset_samples = int(trial_stop_offset_seconds * sfreq)

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=trial_stop_offset_samples,
    preload=True,
)

trial_start_offset_seconds = -0.5
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']

In [4]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split

X_ = []
y_ = []
for X, y, (i_window_in_trial, i_start_sample, i_stop_sample) in windows_dataset:
    X_.append(X)
    y_.append(y)
X_ = torch.as_tensor(X_).float()
y_ = torch.as_tensor(y_)

real_set = TensorDataset(X_, y_)

X_ = torch.from_numpy(np.expand_dims(X_, axis=1))

gen_set = TensorDataset(X_, y_)
gen_loader = DataLoader(dataset=gen_set, batch_size=576, shuffle=True)

print(X_.shape)
print(y_.shape)

torch.Size([576, 1, 22, 1125])
torch.Size([576])


/home/matias/Mestrado/venv_BCI/venv_BCI/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  # This is added back by InteractiveShellApp.init_path()


## Definição do modelo do VAE

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

In [6]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(22*1125, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 22*1125)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 22*1125))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [7]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 22*1125), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [9]:
def train(epoch):
    model.train()
    train_loss = 576
    for batch_idx, (data, _) in enumerate(gen_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(gen_loader.dataset),
                100. * batch_idx / len(gen_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(gen_loader.dataset)))

In [10]:
def test(epoch):
    model.eval()
    test_loss = 576
    with torch.no_grad():
        for i, (data, _) in enumerate(gen_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], recon_batch.view(576, 1, 22, 1125)[:n]])

    test_loss /= len(gen_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [11]:
for epoch in range(1, 51):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(576, 20).to(device)
        sample = model.decode(sample).cpu()

Train Epoch: 1 [0/576 (0%)]	Loss: 17545.581597
====> Epoch: 1 Average loss: 17546.5816
====> Test set loss: 1867217924552.1111
Train Epoch: 2 [0/576 (0%)]	Loss: 1867217924551.111084
====> Epoch: 2 Average loss: 1867217924552.1111
====> Test set loss: 930441671824726.3750
Train Epoch: 3 [0/576 (0%)]	Loss: 930441612172401.750000
====> Epoch: 3 Average loss: 930441612172402.7500
====> Test set loss: 10052464399505367040.0000
Train Epoch: 4 [0/576 (0%)]	Loss: 10052464399505367040.000000
====> Epoch: 4 Average loss: 10052464399505367040.0000
====> Test set loss: 23486066585501696.0000
Train Epoch: 5 [0/576 (0%)]	Loss: 23486064676627344.000000
====> Epoch: 5 Average loss: 23486064676627344.0000
====> Test set loss: 1024033679.2222
Train Epoch: 6 [0/576 (0%)]	Loss: 1024034816.000000
====> Epoch: 6 Average loss: 1024034817.0000
====> Test set loss: 1205589537224.1111
Train Epoch: 7 [0/576 (0%)]	Loss: 1205589304206.222168
====> Epoch: 7 Average loss: 1205589304207.2222
====> Test set loss: 1559

## Processamento dos dados sintéticos para classificação

In [12]:
sample.shape

torch.Size([576, 24750])

In [13]:
size=(22, 1125)
fake = sample.detach().cpu().view(-1, *size)
fake.shape

torch.Size([576, 22, 1125])

In [14]:
fake_set = TensorDataset(fake, y_)

In [15]:
from torch.utils.data import random_split

fake_fulltrainset, fake_evalset = random_split(fake_set, [288, 288])
fake_trainset, fake_testset = random_split(fake_fulltrainset, [192, 96])
real_fulltrainset, real_evalset = random_split(real_set, [288, 288])
real_trainset, real_testset = random_split(real_fulltrainset, [192, 96])

## Definição do modelo do classificador

In [16]:
from braindecode.util import set_random_seeds
from braindecode.models import EEGNetv4

cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = False

seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
n_chans = 22
input_window_samples = 1125
F1, D = 4, 2
kernel_length = 64

model = EEGNetv4(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
    F1=8,
    D=2,
    F2=F1*D,
    kernel_length=kernel_length,
    drop_prob=0.5
)
model.to(device);

## Treinamento do classificador com dados reais

In [17]:
from braindecode import EEGClassifier

batch_size = 32
n_epochs = 50

real_clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=predefined_split(real_testset),
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
real_clf.fit(real_trainset, y=None, epochs=n_epochs);

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.4427        1.4184            0.4062        1.3451  0.0100  1.2378
      2            0.3333        1.2695            0.2188        1.5929  0.0100  0.0844
      3            0.3542        1.0458            0.2292        2.4482  0.0100  0.0833
      4            0.3490        0.9074            0.2292        3.5232  0.0099  0.0778
      5            0.3490        0.7536            0.2292        4.4249  0.0098  0.0774
      6            0.3542        0.6959            0.2292        4.9226  0.0097  0.0762
      7            0.3854        0.5595            0.2396        4.4832  0.0096  0.0767
      8            0.3958        0.5742            0.2500        4.9524  0.0095  0.0773
      9            0.3958        0.5449            0.2604        4.8418  0.0094  0.0750
     10            0.4479       

In [18]:
print(f"Mean Accuracy: {np.mean(real_clf.predict(real_evalset) == [y for X,y in real_evalset])*100:.2f}%")

Mean Accuracy: 79.51%


In [19]:
print(f"Mean Accuracy: {np.mean(real_clf.predict(fake_evalset) == [y for X,y in fake_evalset])*100:.2f}%")

Mean Accuracy: 23.61%


## Treinamento do classificador com dados sintéticos

In [20]:
fake_clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.Adam,
    train_split=predefined_split(fake_testset),
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)
fake_clf.fit(fake_trainset, y=None, epochs=n_epochs);

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.2917        3.5039            0.2083        6.8457  0.0100  0.0871
      2            0.2917        2.0838            0.2083        1.7949  0.0100  0.0853
      3            0.2917        1.6909            0.2083        1.6415  0.0100  0.0818
      4            0.2708        1.6136            0.2917        1.5484  0.0099  0.0745
      5            0.2448        1.5297            0.2604        1.4694  0.0098  0.0736
      6            0.2500        1.4346            0.2708        1.4452  0.0097  0.0745
      7            0.2448        1.4464            0.2604        1.4683  0.0096  0.0735
      8            0.2448        1.3993            0.2604        1.4440  0.0095  0.0733
      9            0.2448        1.3675            0.2604        1.4308  0.0094  0.0745
     10            0.2448       

In [21]:
print(f"Mean Accuracy: {np.mean(fake_clf.predict(real_evalset) == [y for X,y in real_evalset])*100:.2f}%")

Mean Accuracy: 35.07%


In [22]:
print(f"Mean Accuracy: {np.mean(fake_clf.predict(fake_evalset) == [y for X,y in fake_evalset])*100:.2f}%")

Mean Accuracy: 25.35%


## Distância euclidiana entre os dados reais e sintéticos

In [23]:
size=(22, 1125)
real = X_.view(-1, *size)

In [24]:
torch.mean(torch.cdist(real, fake, p=2))

tensor(113.4993)